# Hydrogel-Cell Interaction Test Case

In this case, we only have the deformed shape of the spherical hydrogel as shown in the next few cells.

In [ ]:
import numpy as np
import scipy as sp
import scipy.sparse as spm
from scipy.sparse.linalg import lsqr, spsolve
from scipy.interpolate import RectBivariateSpline
from scipy.optimize import minimize, Bounds, brute, approx_fprime
from scipy.io import loadmat, savemat

import matplotlib.pyplot as plt
from matplotlib import cm, colors
from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Line3DCollection
import sys, time, os.path
from itertools import permutations
import pyshtools

sys.path.append('..')
from shelastic.shutil import SphCoord_to_CartCoord, CartCoord_to_SphCoord, GLQCartCoord
from shelastic.shutil import SHCilmToVector, SHVectorToCilm, SHVec2mesh, SHmesh2Vec
from shelastic.shutil import TransMat, l_coeffs, m_coeffs, LM_list, lmk2K, K2lmk
from shelastic.shbv import generate_submat, Uvec2Tvec, Tvec2Uvec
from shelastic.shvis  import plotfv, vismesh, visSHVec, visSH3d
from sphere_utils import loadCoeffs, arbitrary_force, Ur_interp, usurf2vec, calculateTfv

## Input information

In [ ]:
############################# change inputs here #################################
# Data file name
datadir = ''
smoothed = ''
dilated = ''
shapename = 'noise'
datafile = os.path.join(datadir, shapename+'_Coordinates_Cart_'+smoothed+'.csv')
connfile = os.path.join(datadir, shapename+'_Connectivity.csv')
maskfile = os.path.join(datadir, shapename+'_Mask'+dilated+'.csv')

# Material properties
mu0 = 300/3; nu0 = 0.499;
r0 = 5;           # microns

# Arbitrary force field (opposite forces)
pF = 1            # force magnitude (Pa)
rcut_r0 = 0.3; 
rcut = rcut_r0 * r0
mag = 4; dilation = 2;
noise_level = 0.01 # noise std (um)

# Spherical Harmonics Analysis Settings
lJmax = 20; lKmax = lJmax; lmax_plot = 60;

# initial guess settings
init_guess_type = 'ur-only' #'ur-only' or 'true'

# regularizations
myalpha = 1     # traction magnitude
mybeta  = 1  # coefficient magnitude

# program switches
plot_figure = False
myord = 1; # myord: p-norm order (1-mean value)

# minimization settings
N_period = 100
maxiter_per_period = 5
CG_gtol = 1e-5

minimizer = 'CG'
minimizer_config = {'maxiter': maxiter_per_period, 'disp': True, 'gtol': CG_gtol}

# dump files for minimization
savename = shapename + ('_b%.0e_r%.1f_n%.1e_lmax%d'%(mybeta, rcut_r0, noise_level, lJmax))
#+('_a%.0e_b%.0e'%(myalpha, mybeta))+('_lmax%d'%lJmax)+smoothed+dilated

# settings for loading \hat{U}, \hat{T} coefficients
Cmat, Dmat = loadCoeffs(mu0, nu0, lJmax, 'reg')

## Define arbitrary force field

In [ ]:
Tvec_true, Uvec_true, Vp, Tfv = arbitrary_force(rcut_r0) #, noise_level=noise_level, Cmat=Cmat, Dmat=Dmat)

if plot_figure:
    fig, ax = plt.subplots(1, 1)
    ax.plot(np.abs(Tvec_true))
    ax.set_title('Traction SH coefficients $T^{(K)}$')
    plt.show()

In [ ]:
if plot_figure:
    fig, ax = visSHVec(Uvec_true*r0, lmax_plot=lmax_plot, SphCoord=True, Complex=True,
                       config_quiver=(2, 3, 'k', 10), lonshift=180, figsize=(6,3))
    fig, ax = plt.subplots()
    ax.plot(np.abs(Uvec_true))
    ax.set_title('Displacement SH Coefficients $U^{(K)}$')
    plt.show()

In [ ]:
if plot_figure:
    umesh_fine_scaled = SHVec2mesh(Uvec_true, lmax=lmax_plot, SphCoord=False, Complex=True)
    tmesh_fine = SHVec2mesh(Tvec_true*mu0, lmax=lmax_plot, SphCoord=False, Complex=True)
    fig, ax = visSH3d(umesh_fine_scaled, cmesh=tmesh_fine, r0=r0, show=False, 
                      elevation=0, azimuth=0, surface=True, figsize=plt.figaspect(1))
    plt.show()

In [ ]:
#### Plot the geometry (Vp) ####
if plot_figure:
    fig = plt.figure(figsize=plt.figaspect(1))
    ax = fig.add_subplot(111, projection='3d')

    Tf_id = Tfv > 0.1
    nTf_id= np.logical_not(Tf_id)
    ax.scatter3D(Vp[Tf_id, 0], Vp[Tf_id, 1], Vp[Tf_id, 2])
    ax.scatter3D(Vp[nTf_id, 0], Vp[nTf_id, 1], Vp[nTf_id, 2])

    ax.view_init(azim=0, elev=30)
    plt.show()

Then we determine the original radius of the particle, assuming the particle is incompressible. The total volume can be estimated by adding the volume of the tetrahedrons. The volume of a tetrahedron is calculated as:

$$
V_{0123}=\frac{1}{6}
\begin{vmatrix}
 x_1 & y_1 & z_1 & 1\\ 
 x_2 & y_2 & z_2 & 1\\ 
 x_3 & y_3 & z_3 & 1\\ 
 0 & 0 & 0 & 1\\ 
\end{vmatrix}
$$

We need to solve the reverse problem of a deformed shape. We will try the following methods to tackle this problem:

1. Assume $r$-direction deformation only, using the solution as initial guess to the optimization
2. LSQ solving coeffs of the SH solutions for fitting both the traction-free boundary and the shape

Conversion between complex and real spherical harmonics, for $m>0$

$$
f_{lm} = \left[f_l^m+(-1)^mf_l^{-m}\right]/\sqrt{2}\\
f_{l-m}=i\left[f_l^m-(-1)^mf_l^{-m}\right]/\sqrt{2}\\
f_l^m = (f_{lm}-if_{l-m})/\sqrt{2}\\
f_l^{-m} = (-1)^m(f_{lm}+if_{l-m})/\sqrt{2}
$$

for $m=0$:

$$
f_{l0} = f_l^0
$$


## 1. LSQ solving SH coeffs for displacement field

Obviously, the decomposition is not satisfactory. It is not reasonable to assume the deformation is only on $r$-direction. In this section, we will try to optimize SH coeffs, so that the deformed shape is closest to the data. Notice that the integral of a spherical harmonic function on the sphere surface is:

$$
\int_0^{2\pi}\!\int_0^{\pi}Y_l^m(\theta,\varphi)\sin\theta d\theta d\varphi = 4\pi\delta_{l0}\delta_{m0}
$$

Therefore, only the $Y_0^0$ term controls the rigid body translation (constant). If we only impose higher mode spherical harmonics, there will be no rigid body motion.

## 2. Obtain the shape interpolation function

A function that returns $u_r(\theta,\varphi)$

In [ ]:
f_interp = Ur_interp(Vp/r0, lmax=lJmax+10, plot_figure=plot_figure)

## 3. Target function including traction residual and elastic energy

We use the true solution we obtained from Section 1 `Tvec_true` for the testing. Load the initial guess as:

In [ ]:
import glob

filelist = glob.glob('AK_'+savename+'_??.npz')
nfile = len(filelist)
if nfile > 0:
    u0_surf = np.load('AK_'+savename+'_%02d.npz'%(nfile-1))['u_surf_list'][-1]
elif init_guess_type == 'true':
    u0_vec = Uvec.copy()
    x0mesh = SHVec2mesh(u0_vec, lmax=lJmax, SphCoord=False, Complex=True) + X0/r0
    r0surf, t0surf, p0surf = CartCoord_to_SphCoord(x0mesh[..., 0], x0mesh[..., 1], x0mesh[..., 2])
    lat0surf = 90 - np.rad2deg(t0surf); lon0surf = np.rad2deg(p0surf);
    u0_surf = np.stack([lat0surf, lon0surf], axis=-1) - X0shear
elif init_guess_type == 'ur-only':
    u0_surf = np.zeros((lJmax+1, lJmax*2+1, 2))
print(u0_surf.shape)

U0vec, aK, T0vec = usurf2vec(u0_surf, f_interp=f_interp, lmax=lJmax)
if plot_figure:
    fig, ax = plt.subplots()
    ax.plot(np.abs(T0vec))
    ax.set_title('Traction SH Coefficients $T^{(K)}$')
    plt.show()
    #     print('Displacement in Spherical Coordinates...')
    #     fig, ax = visSHVec(U0vec*r0, lmax_plot=lmax_plot, SphCoord=True, Complex=True,
    #                        n_vrange=(-1, 1), s_vrange=(0, 1),
    #                        config_quiver=(2, 3, 'k', 10), lonshift=180, figsize=(6,3))
    # print('Traction in Spherical Coordinates...')
    figs, ax = visSHVec((T0vec)*mu0, lmax_plot=lmax_plot, SphCoord=True, Complex=True, 
                       n_vrange=(-100, 100), s_vrange=(0, 50), show=False,
                       config_quiver=(2, 3, 'k', 500), lonshift=180, figsize=(6,3))
    ax[0].set_title('normal traction $T_n$')
    ax[1].set_title('shear traction $T_s$')

In [ ]:
# Define weights and traction free region
if dilated == '_softedge':
    isTfv = calculateTfv(U0vec, lJmax, Vp/r0, mask)
else:
    isTfv = calculateTfv(U0vec, lJmax, Vp/r0, Tfv)

print(isTfv.shape)

# Define shape reference
latsdeg, lonsdeg = pyshtools.expand.GLQGridCoord(lJmax)
lon0, lat0 = np.meshgrid(lonsdeg, latsdeg)
X0surf = np.stack([lat0, lon0], axis=-1)
X0 = GLQCartCoord(lJmax)

In [ ]:
from sphere_utils import usurf2vec, Tvec2Tres, usurf2dr
T0dist = Tvec2Tres(T0vec, lJmax, isTfv=isTfv, norm_order=myord)
E0el = np.vdot(U0vec, T0vec).real*2*np.pi
print('Traction residual: %.4e Pa'%(T0dist*mu0))
print('Elastic energy: %.4e pJ'%(E0el*(r0/1e6)**3*mu0*1e12))
print('funval: %.4e %.4e'%(T0dist, E0el))

In [ ]:
target_args = (f_interp, lJmax, mybeta, myord, X0surf, X0, isTfv, Cmat, Dmat)

def print_iter(xk):
    Uvec, aK, Tvec = usurf2vec(xk, f_interp, lJmax, X0surf=X0surf, X0=X0, Cmat=Cmat, Dmat=Dmat)
    Tdist = Tvec2Tres(Tvec, lJmax, isTfv=isTfv, norm_order=myord)
    Eel = np.vdot(Uvec, Tvec).real*2*np.pi
    dr  = usurf2dr(xk, *target_args)
    print('%13.4ePa%13.4epJ%13.4e%13.4e%13.4e'%(Tdist*mu0, Eel*(r0/1e6)**3*mu0*1e12, Tdist, Eel, dr))

bd = Bounds(-6, 6)
u_surf = u0_surf.copy()
u_surf_list = [u_surf, ]
for i in range(N_period):
    print('Period %d'%i)
    u_res = minimize(usurf2dr, u_surf.flatten(), args=target_args, method = minimizer,
                     options=minimizer_config, callback=print_iter)
    if u_res.success:
        break
    u_surf = u_res.x.copy()
    u_surf_list.append(u_surf)

In [ ]:
Usurfvec, aK, Tsurfvec = usurf2vec(u_surf, f_interp, lJmax, X0surf=X0surf, X0=X0, Cmat=Cmat, Dmat=Dmat)
Tsurfdist = Tvec2Tres(Tsurfvec, lJmax, isTfv=isTfv, norm_order=myord)
Esurfel = np.vdot(Usurfvec, Tsurfvec).real*2*np.pi
print('Traction residual: %.4e Pa'%(Tsurfdist*mu0))
print('Elastic energy: %.4e pJ'%(Esurfel*(r0/1e6)**3*mu0*1e12))
print('funval: %.4e %.4e'%(Tsurfdist, Esurfel))

In [ ]:
print(u_res)
u_surf = u_res.x

if plot_figure:
    fig, ax = plt.subplots()
    ax.plot(np.abs(Tvec_true), label='True solution')
    ax.plot(np.abs(Tsurfvec),  label='Reconstructed')
    ax.set_title('Traction SH coefficients')
    plt.show()

    #     print('Displacement in Spherical Coordinates...')
    #     fig, ax = visSHVec(Usurfvec*r0, lmax_plot=lmax_plot, SphCoord=True, Complex=True, 
    #                        s_vrange=(0,0.01), show=False,
    #                        config_quiver=(2, 3, 'k', 10), lonshift=180, figsize=(6,3))
    print('Traction in Spherical Coordinates...')
    fig, ax = visSHVec((Tsurfvec-Tvec_true)*mu0, lmax_plot=lmax_plot, SphCoord=True, Complex=True, 
                       n_vrange=(-100, 100), s_vrange=(0, 50), show=False,
                       config_quiver=(2, 3, 'k', 500), lonshift=180, figsize=(6,3))
    ax[0].set_title('Normal traction error $|T_n - T_n^{true}|$')
    ax[1].set_title('Shear traction error $|T_s - T_s^{true}|$')


In [ ]:
import glob
filelist = glob.glob('AK_'+savename+'_??.npz')
nfile = len(filelist)
np.savez_compressed(('AK_'+savename+'_%02d.npz')%(nfile), AK_iter=aK, u_surf_list=u_surf_list, beta=mybeta, N=N_period)